2024-08-19


[자동차 평균 대여 기간 구하기](https://school.programmers.co.kr/learn/courses/30/lessons/157342)


```sql

-- 실패
-- END_DATE - START_DATE가 두 날짜 간의 차이를 계산할 때 DBMS에 따라 날짜 차이를 일수로 반환하지 않을 수 있습니다.
-- '2022-09-01'에서 '2022-08-01'을 빼면 31이 아니라 100이 나옵니다.

SELECT  CAR_ID, ROUND(AVG(END_DATE - START_DATE), 2) AVERAGE_DURATION
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
GROUP BY CAR_ID
HAVING AVG(END_DATE - START_DATE) >= 7
ORDER BY AVERAGE_DURATION DESC, CAR_ID DESC;

-- 실패
SELECT 
CAR_ID, 
ROUND(AVG(DATEDIFF(END_DATE, START_DATE)), 1) AS AVERAGE_DURATION
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
GROUP BY CAR_ID
HAVING AVERAGE_DURATION >= 7
ORDER BY 
AVERAGE_DURATION DESC, 
CAR_ID DESC;

-- (DATEDIFF(END_DATE, START_DATE)
-- +1 이슈 -> 성공
-- END_DATE + 1이 정상적으로 동작하는 DBMS에서만 사용 가능합니다. 

SELECT 
CAR_ID, 
ROUND(AVG(DATEDIFF(END_DATE, START_DATE)), 1)+1 AS AVERAGE_DURATION
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
GROUP BY CAR_ID
HAVING AVERAGE_DURATION >= 7
ORDER BY 
AVERAGE_DURATION DESC, 
CAR_ID DESC;
```

[조회수가 가장 많은 중고거래 게시판의 첨부파일 조회하기](https://school.programmers.co.kr/learn/courses/30/lessons/164671)


```sql
SELECT CONCAT('/home/grep/src/', UGB.BOARD_ID, '/', UGF.FILE_ID, UGF.FILE_NAME, UGF.FILE_EXT) FILE_PATH
FROM USED_GOODS_BOARD UGB
JOIN USED_GOODS_FILE UGF
ON UGF.BOARD_ID = UGB.BOARD_ID
WHERE UGB.VIEWS = (SELECT MAX(VIEWS) FROM USED_GOODS_BOARD)
ORDER BY UGF.FILE_ID DESC;


-- 윈도우 함수 버전 : RANK() OVER

WITH MaxViews AS (
    SELECT 
        BOARD_ID, 
        VIEWS,
        RANK() OVER (ORDER BY VIEWS DESC) AS view_rank
    FROM 
        USED_GOODS_BOARD
)
SELECT 
    CONCAT('/home/grep/src/', UGB.BOARD_ID, '/', UGF.FILE_ID, UGF.FILE_NAME, UGF.FILE_EXT) AS FILE_PATH
FROM 
    MaxViews UGB
JOIN 
    USED_GOODS_FILE UGF ON UGF.BOARD_ID = UGB.BOARD_ID
WHERE 
    UGB.view_rank = 1
ORDER BY 
    UGF.FILE_ID DESC;

```
[자동차 대여 기록 별 대여 금액 구하기](https://school.programmers.co.kr/learn/courses/30/lessons/151141)

<!-- REGEXP_REPLACE(P.DURATION_TYPE, '[^0-9]', '') -->

<!-- 실패 -->
```sql

WITH TRUCK AS (
    SELECT *
    FROM CAR_RENTAL_COMPANY_CAR
    WHERE CAR_TYPE = '트럭'
)

SELECT 
H.HISTORY_ID,
ROUND(CASE 
WHEN DATEDIFF(end_date, start_date) + 1 >= 90 
THEN TRUCK.DAILY_FEE * (SELECT (1-DISCOUNT_RATE/100) FROM CAR_RENTAL_COMPANY_DISCOUNT_PLAN WHERE REGEXP_REPLACE(DURATION_TYPE, '[^0-9]', '') = '90' AND CAR_TYPE = '트럭') * (DATEDIFF(end_date, start_date) + 1)
WHEN DATEDIFF(end_date, start_date) + 1 >= 30 
THEN TRUCK.DAILY_FEE * (SELECT (1-DISCOUNT_RATE/100) FROM CAR_RENTAL_COMPANY_DISCOUNT_PLAN WHERE REGEXP_REPLACE(DURATION_TYPE, '[^0-9]', '') = '30' AND CAR_TYPE = '트럭') * (DATEDIFF(end_date, start_date) + 1)
WHEN DATEDIFF(end_date, start_date) + 1 >= 7 
THEN TRUCK.DAILY_FEE * (SELECT (1-DISCOUNT_RATE/100) FROM CAR_RENTAL_COMPANY_DISCOUNT_PLAN WHERE REGEXP_REPLACE(DURATION_TYPE, '[^0-9]', '') = '7' AND CAR_TYPE = '트럭') * (DATEDIFF(end_date, start_date) + 1) ELSE 
TRUCK.DAILY_FEE * (DATEDIFF(end_date, start_date) + 1)
END) AS FEE
FROM 
    CAR_RENTAL_COMPANY_RENTAL_HISTORY H
JOIN 
    TRUCK ON TRUCK.CAR_ID = H.CAR_ID
ORDER BY FEE DESC, HISTORY_ID DESC

```

[조건에 부합하는 중고거래 상태 조회하기](https://school.programmers.co.kr/learn/courses/30/lessons/164672)

```sql
-- 코드를 입력하세요
SELECT BOARD_ID, WRITER_ID, TITLE, PRICE, 
CASE STATUS
WHEN 'SALE' THEN '판매중'
WHEN 'RESERVED' THEN '예약중'
WHEN 'DONE' THEN '거래완료'
END AS "STATUS"
FROM USED_GOODS_BOARD
WHERE DATE_FORMAT(CREATED_DATE, '%Y-%m-%d') = '2022-10-05'
ORDER BY 1 DESC;

```

[자동차 대여 기록에서 장기/단기 대여 구분하기](https://school.programmers.co.kr/learn/courses/30/lessons/151138)

```sql

SELECT HISTORY_ID, CAR_ID, DATE_FORMAT(START_DATE, '%Y-%m-%d') START_DATE,  DATE_FORMAT(END_DATE, '%Y-%m-%d') END_DATE, IF(DATEDIFF(end_date,start_date)+1 >= 30, '장기 대여', '단기 대여') RENT_TYPE
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
WHERE DATE_FORMAT(start_date, '%Y-%m') = '2022-09'
ORDER BY HISTORY_ID DESC;
```